In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import torch

from opf.test import load_run, test_run

mpl.rcParams["text.usetex"] = False

sns.set_style("white")
sns.set_context("paper")

torch.set_float32_matmul_precision("high")

In [ ]:
import wandb

api = wandb.Api()
runs = api.runs(
    "damowerko-academic/opf",
    filters={"$and": [{"config.case_name": "case300_ieee_simple"}, {"tags": "hybrid"}]},
)

In [ ]:
runs.histories(format="pandas").groupby("run_id")["val/invariant"].min()

In [ ]:
run_ids = {
    "IEEE 30": {
        "Supervised": "bzusmsf4",  # 1000 epochs
        "Augmented": "",
        "Pointwise": "qlq6szw2",  # 2000 epochs
        "Hybrid": "3z34izuj",  # 1000 epochs
    },
    "IEEE 57": {
        "Supervised": "vxepjaw9",  # 1000 epochs
        "Augmented": "phjzbws9",  # 1000 epochs
        "Pointwise": "bf1uux9g",  # 2000 epochs
        "Hybrid": "pgsf77xl",  # 1000 epochs
    },
    "IEEE 118": {
        "Supervised": "vjn6wwkb",  # Old run
        "Augmented": "bzusmsf4",  # Old run
        "Pointwise": "w50zi687",  # Old run
        "Hybrid": "jmzzicbk",  # 1000 epochs, old version
    },
    "IEEE 300": {
        "Supervised": "2as4y9kv",  # Old run
        "Augmented": "ig4u3u49",  # Old run
        "Pointwise": "hy3d6jy0",  # 2000 epochs
        "Hybrid": "2as4y9kv",  # Old run
    },
    "PEGASE 1354": {
        "Supervised": "",
        "Augmented": "",
        "Pointwise": "",
        "Hybrid": "",
    },
}

In [ ]:
dfs = []
for case_name in run_ids:
    for model_name, id in run_ids[case_name].items():
        if id == "":
            continue
        df = test_run(id, load_existing=False, project=True, clamp=True)
        df = df.assign(id=id, case_name=case_name, model_name=model_name)
        dfs.append(df)

df = pd.concat(dfs)

In [ ]:
index = pd.MultiIndex.from_product(
    [
        ["IEEE 30", "IEEE 57", "IEEE 118", "IEEE 300", "PEGASE 1354"],
        ["Supervised", "Augmented", "Pointwise", "Hybrid"],
    ],
    names=["Case", "Model"],
)

In [ ]:
df["optimality_gap"] = df["test/cost"] / df["acopf/cost"] - 1
df.groupby(["case_name", "model_name"])[
    [
        "optimality_gap",
        "test/inequality/error_mean",
        "test/inequality/error_max",
        "test/equality/error_mean",
        "test/equality/error_max",
    ]
].mean().reindex(index).style.format("{:.2%}", na_rep="--").set_table_styles(
    [
        {
            "selector": f"tr:nth-child({i * 4})",
            "props": [("border-bottom", "2px solid black")],
        }
        for i in range(1, 5)
    ]
)

In [ ]:
import re

constraint_pattern = re.compile(
    r"test/(equality|inequality)/([^/]+)/(error_mean|error_max)"
)
metric_columns = {}
for column in df.columns:
    match = constraint_pattern.match(column)
    if not match:
        continue
    _, constraint_name, error_type = match.groups()
    error_type = error_type.replace("error_", "")
    metric_columns[column] = (constraint_name, error_type)

id_vars = ["case_name", "model_name"]
df_melted = df[id_vars + list(metric_columns.keys())].melt(
    id_vars=id_vars,
    value_vars=list(metric_columns.keys()),
)
df_melted["constraint_type"] = df_melted["variable"].apply(
    lambda x: metric_columns[x][0]
)
df_melted["statistic"] = df_melted["variable"].apply(lambda x: metric_columns[x][1])
df_melted = df_melted.drop(columns=["variable"])

df_pivoted = (
    df_melted.groupby(id_vars + ["constraint_type", "statistic"])
    .mean()
    .pivot_table(
        index=id_vars,
        columns=["constraint_type", "statistic"],
        values="value",
    )
    .reindex(index)
)
df_pivoted.style.format("{:.2%}", na_rep="--").set_table_styles(
    [
        {
            "selector": f"tr:nth-child({i * 4})",
            "props": [("border-bottom", "2px solid black")],
        }
        for i in range(1, 5)
    ]
)